In [47]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import pandas as pd
from sqlalchemy import create_engine

In [40]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [41]:
br_paintings_url = 'https://www.twoinchbrush.com/all-paintings'
browser.visit(br_paintings_url)
time.sleep(1)

br_soup = bs(browser.html,'html.parser')

In [42]:
painting_holder = br_soup.body.find('div', id='holder')
paintings = painting_holder.find_all('div', class_='painting-holder')

In [44]:
painting_data = []

for painting in paintings:
    episode_id = painting.find('p', class_='details').text.strip().split('\n')[0]
    episode_split = episode_id.split(' Episode ')
    
    season = episode_split[0].split('Season ')[1]
    if len(season) == 1:
        season = f'0{season}'
    else:
        season
    
    episode = episode_split[1]
    if len(episode) == 1:
        episode = f'0{episode}'
    else:
        episode
    
    episode_id = f'S{season}E{episode}'
    
    episode_name = painting.find('p').text
    img_url = f'https://www.twoinchbrush.com{painting.find("a", class_="progressive")["data-img"]}'
    
    painting_link = f'https://www.twoinchbrush.com{painting.find("a", class_="progressive")["href"]}'
    browser.visit(painting_link)
    time.sleep(1)
    
    painting_page_soup = bs(browser.html,'html.parser')
    video_embed = painting_page_soup.find('div', class_='videoWrapper')
    video_url = video_embed.find('iframe')['src']
    
    painting_info = {"episode_id":episode_id,
                    "episode_name":episode_name,
                    "img_url":img_url,
                    "video_url": video_url}
    
    painting_data.append(painting_info)



[{'episode_id': 'S01E01', 'episode_name': 'A Walk in the Woods', 'img_url': 'https://www.twoinchbrush.com/images/painting282.png', 'video_url': 'https://www.youtube.com/embed/oh5p5f5_-7A'}, {'episode_id': 'S01E02', 'episode_name': 'Mt. McKinley', 'img_url': 'https://www.twoinchbrush.com/images/painting283.png', 'video_url': 'https://www.youtube.com/embed/RInDWhYceLU'}, {'episode_id': 'S01E03', 'episode_name': 'Ebony Sunset', 'img_url': 'https://www.twoinchbrush.com/images/painting284.png', 'video_url': 'https://www.youtube.com/embed/UOziR7PoVco'}, {'episode_id': 'S01E04', 'episode_name': 'Winter Mist', 'img_url': 'https://www.twoinchbrush.com/images/painting285.png', 'video_url': 'https://www.youtube.com/embed/0pwoixRikn4'}, {'episode_id': 'S01E05', 'episode_name': 'Quiet Stream', 'img_url': 'https://www.twoinchbrush.com/images/painting286.png', 'video_url': 'https://www.youtube.com/embed/DFSIQNjKRfk'}, {'episode_id': 'S01E06', 'episode_name': 'Winter Moon', 'img_url': 'https://www.two

TypeError: 'NoneType' object is not subscriptable

In [79]:
painting_df = pd.DataFrame(painting_data)
painting_df.head()

,episode_id,episode_name,img_url,video_url
0,S01E01,A Walk in the Woods,https://www.twoinchbrush.com/images/painting28...,https://www.youtube.com/embed/oh5p5f5_-7A
1,S01E02,Mt. McKinley,https://www.twoinchbrush.com/images/painting28...,https://www.youtube.com/embed/RInDWhYceLU
2,S01E03,Ebony Sunset,https://www.twoinchbrush.com/images/painting28...,https://www.youtube.com/embed/UOziR7PoVco
3,S01E04,Winter Mist,https://www.twoinchbrush.com/images/painting28...,https://www.youtube.com/embed/0pwoixRikn4
4,S01E05,Quiet Stream,https://www.twoinchbrush.com/images/painting28...,https://www.youtube.com/embed/DFSIQNjKRfk


In [80]:
painting_df['episode_id'].value_counts()

S13E12    2
S14E09    2
S18E02    2
S18E12    2
S27E03    2
S03E04    1
S19E06    1
S04E08    1
S02E12    1
S27E10    1
S28E08    1
S20E07    1
S19E07    1
S07E04    1
S12E12    1
S31E01    1
S30E13    1
S25E08    1
S14E01    1
S16E12    1
S27E02    1
S15E03    1
S05E08    1
S23E07    1
S01E01    1
S12E10    1
S25E09    1
S24E01    1
S08E06    1
S18E01    1
         ..
S18E10    1
S29E04    1
S21E12    1
S30E09    1
S12E13    1
S15E02    1
S17E07    1
S07E02    1
S16E10    1
S02E11    1
S29E02    1
S10E06    1
S22E13    1
S05E13    1
S12E07    1
S18E04    1
S24E05    1
S16E03    1
S26E09    1
S15E04    1
S05E04    1
S03E10    1
S10E11    1
S05E06    1
S26E02    1
S06E04    1
S06E09    1
S04E11    1
S01E06    1
S07E07    1
Name: episode_id, Length: 398, dtype: int64

In [81]:
painting_df.loc[(painting_df['episode_name'] == 'Lost Lake'), ['episode_id']] = 'S13E13'
painting_df.loc[(painting_df['episode_name'] == 'On a Clear Day'), ['episode_id']] = 'S14E08'
painting_df.loc[(painting_df['episode_name'] == 'Rippling Waters'), ['episode_id']] = 'S18E13'
painting_df.loc[(painting_df['episode_name'] == 'Mountain Seclusion'), ['episode_id']] = 'S18E03'
painting_df.loc[(painting_df['episode_name'] == 'Wilderness Falls'), ['episode_id']] = 'S27E04'

In [82]:
engine = create_engine("sqlite:///db/bob_ross.sqlite")
painting_df.to_sql('paintings_by_episode', con=engine, if_exists='replace', index=False)